In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import fft
from scipy.signal import get_window
import glob
import time

def plot_scatter_func(x, y, label, title, xlab, ylab, x1 = None, x2 = None, y1 = None, y2 = None):

    f, ax = plt.subplots(figsize = (20, 10))
    ax.scatter(x, y, label = label)
    ax.set_title(title, fontsize = 30)
    ax.set_xlabel(xlab, fontsize = 30)
    ax.set_ylabel(ylab, fontsize = 30)
    ax.tick_params(axis='both', which='major', labelsize=30)
    ax.set_xlim(x1, x2)
    ax.set_ylim(y1, y2)
    ax.legend(loc = 'best', fontsize = 20)
    ax.grid(True)

    return f

def phase_func(Y):
    phases = np.angle(Y) * 180 / np.pi
    
    return phases

#dBu = 20log10(V/0.775V)

#Why the windowing? The FFT assumes that the signal is periodic and continuous within the window. 
#If the signal is not perfectly periodic, it can cause spectral leakage, which results in an offset or differences in the frequency spectrum.

def rfft(y, cnv): #this function computes the fft of y
    #window = get_window("hanning", len(y))
    #window = 1
    #y_windowed = y * window
    if np.isnan(y).any():
        nan_indices = np.where(np.isin(y))[0]
        print(f"Input data contains NaN values in {nan_indices}.")
        
    if np.isinf(y).any():
        print("Input data contains Inf values.")
        
    N = len(y)
    print(f"y_windowed: {y}")
    Y = fft.rfft(y)
    print(f"Y: {Y}")
    phases = phase_func(Y)
    print(phases)
    Y_normalized = np.abs(Y) / N
    Y_dBu = 20*(np.log10(np.abs(Y_normalized)  * cnv / 0.775 ))
        
    return Y_dBu, phases

In [4]:
hour = '15.42'
day = '08082024'

In [7]:
file_pattern = f"../../Waveforms/{day}/{day}_{hour}/{day}_{hour}_wf.csv/{day}_{hour}_wf_*.csv"
file_list = glob.glob(file_pattern)

amplitudes_list = []

if not file_list:
    print("No files found for the given pattern.")
else:
    for i, file in enumerate(file_list):
        df = pd.read_csv(file)
        if i == 0:
            print(df.iloc[:, 0])
            print(df.iloc[:, 1])
            times = pd.to_numeric(df.iloc[1:, 0], errors = 'coerce').values #The argument errors='coerce' specifies that any non-numeric values encountered during the conversion should be replaced with NaN (Not a Number)
        amplitudes = pd.to_numeric(df.iloc[1:, 1], errors='coerce').values
        if np.isnan(amplitudes).any():
            nan_indices = np.where(np.isnan(amplitudes))[0]
            print(f"Input data contains NaN values in {nan_indices} at {i} file.")
            #amplitudes_A[nan_indices] = 0
 
        amplitudes_list.append(amplitudes)

all_amplitudes = np.stack(amplitudes_list, axis = 0)
print(all_amplitudes)

No files found for the given pattern.


ValueError: need at least one array to stack